<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/search_kokkai_speeches_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [9]:
from openai import OpenAI
import requests

In [12]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# クライアントの準備
client = OpenAI()

In [6]:
# 国会議事録APIのエンドポイント
url = "https://kokkai.ndl.go.jp/api/speech"

In [30]:
def search_speeches(query):
    # 検索条件を設定
    params = {
        "any": query,
        "recordPacking": "json",
        "from": "2024-01-01",
        "until": "2024-02-01",
    }

    # APIにリクエストを送信
    response = requests.get(url, params=params)

    # レスポンスをJSONとして解析
    data = response.json()
    print(data)
    # 検索結果の発言を抽出
    speeches = [speech["speech"] for speech in data["speechRecord"]]

    return speeches

In [38]:
def generate_answer(question, context):
    # 質問と関連する発言を組み合わせてプロンプトを作成
    prompt = [
          {
              "role":"user",
              "content":f"質問: {question}に関する発言はありますか？\n\n関連する国会議事録の発言:\n" + "\n".join(context) + "\n\n回答:"
          }]

    # OpenAIのAPIを使用して回答を生成
    response = client.chat.completions.create(
      model="gpt-4",
      messages=prompt,
      temperature=0.2,
      max_tokens=500
    )

    # 生成された回答を返す
    answer = response.choices[0].message.content
    return answer

In [39]:
def main():
    while True:
        # ユーザーからの質問を受け取る
        question = input("質問を入力してください（終了するには'q'を入力）: ")

        if question.lower() == 'q':
            break

        # 関連する発言を検索
        # responseが受け取るtoken数を超えるとエラーになります
        context = search_speeches(question)

        if not context:
            print("関連する発言が見つかりませんでした。")
            continue

        # 回答を生成
        answer = generate_answer(question, context)

        # 回答を出力
        print(f"回答: {answer}\n")

if __name__ == "__main__":
    main()

質問を入力してください（終了するには'q'を入力）: トラックGメン
{'numberOfRecords': 1, 'numberOfReturn': 1, 'startRecord': 1, 'nextRecordPosition': None, 'speechRecord': [{'speechID': '121315254X00320240201_003', 'issueID': '121315254X00320240201', 'imageKind': '会議録', 'searchObject': 3, 'session': 213, 'nameOfHouse': '参議院', 'nameOfMeeting': '本会議', 'issue': '第3号', 'date': '2024-02-01', 'closing': None, 'speechOrder': 3, 'speaker': '岸田文雄', 'speakerYomi': 'きしだふみお', 'speakerGroup': '自由民主党・無所属の会', 'speakerPosition': '内閣総理大臣', 'speakerRole': None, 'speech': '○内閣総理大臣（岸田文雄君）\u3000水岡俊一議員の御質問にお答えいたします。\r\n\u3000能登半島における地震の長期評価についてお尋ねがありました。\r\n\u3000地震調査委員会では、活断層や海溝沿いで起きる地震について、地震活動、地殻変動、地質等の調査データに基づいて、それぞれの調査の進捗等に応じ、順次長期評価を行っていると承知をしています。日本海側の海域活断層については、令和四年の九州・中国地方沖の評価の公表に続いて、必要なデータの整理、分析をした上で、現在、能登地方沖を含む海域の評価を進めているところです。\r\n\u3000能登地方の地震活動が継続している状況に鑑み、能登地方の海域活断層の長期評価を早急に進め、その結果を公表するとともに、必要な情報発信等を行ってまいります。\r\n\u3000北陸地方における道路啓開計画の策定についてお尋ねがありました。\r\n\u3000道路啓開計画については、首都直下及び南海トラフなどの科学的に発生確率が高い大規模地震から順次策定してきており、北陸地方においても策定に